<a href="https://colab.research.google.com/github/felanbi/Projet-OC-Categoriser-automatiquement-des-questions/blob/main/P5_02_notebooktest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import nltk
import pickle

In [3]:
data = pd.read_pickle("drive/MyDrive/data/data_tokenized_sw_free_v2.pkl")
data.head()

,post,tags
Id,,
65526399,node taking hour employee recordsgroovy xmlbel...,[javascript]
65526401,pythonuser usingibased game game begin create ...,[python]
65526407,remove ver powered whmcomplete solution whmcsi...,"[php, css]"
65526419,object allocated allocate constructremoved des...,[c++]
65526447,react route change createbrowserhistory blockr...,"[javascript, reactjs]"


In [4]:
df = data.copy()
df.post = df.post.str.split()
df.head()

,post,tags
Id,,
65526399,"[node, taking, hour, employee, recordsgroovy, ...",[javascript]
65526401,"[pythonuser, usingibased, game, game, begin, c...",[python]
65526407,"[remove, ver, powered, whmcomplete, solution, ...","[php, css]"
65526419,"[object, allocated, allocate, constructremoved...",[c++]
65526447,"[react, route, change, createbrowserhistory, b...","[javascript, reactjs]"


In [5]:
df.shape

(296944, 2)

In [8]:
vocabulary = pickle.load(open("drive/MyDrive/data/vocabulary.pkl", "rb"))

In [ ]:
from gensim.models import TfidfModel
import gensim.corpora as corpora
from gensim import models
from gensim.utils import simple_preprocess
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import matplotlib.pyplot as plt

texts = df.post.to_list()
# id2word = corpora.Dictionary(texts)
# id2word.filter_extremes(keep_n = 10000)

id2word = corpora.Dictionary()
id2word.add_documents([[x] for x in vocabulary])

bow_corpus = [id2word.doc2bow(text) for text in texts]
tfidf = TfidfModel(bow_corpus)
tfidf_corpus = [tfidf[text] for text in bow_corpus]
# appliquer ACP

model = LdaMulticore(corpus = tfidf_corpus,
                    id2word = id2word,
                    num_topics = 20, # nombre de tags retenus
                    random_state = 42,
                    passes = 10,
                    workers = 7,
                    minimum_probability = 0)

coherencemodel = CoherenceModel(model = model, 
                                texts = texts,
                                dictionary = id2word, 
                                coherence = 'c_v')

coherencemodel.get_coherence()

0.5836768255950322

In [ ]:
from pprint import pprint

print('\nPerplexity: ', model.log_perplexity(tfidf_corpus))

coherence_model_lda = CoherenceModel(model=model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

model_topics = model.show_topics(formatted=False)
pprint(model.print_topics(num_words=30))


Perplexity:  -9.32708830006051

Coherence Score:  0.5836768255950322
[(0,
  '0.022*"react" + 0.019*"const" + 0.016*"component" + 0.008*"app" + '
  '0.007*"page" + 0.007*"log" + 0.006*"route" + 0.006*"usestate" + '
  '0.006*"data" + 0.006*"useeffect" + 0.006*"div" + 0.006*"prop" + 0.006*"vue" '
  '+ 0.005*"type" + 0.005*"props" + 0.005*"import" + 0.005*"error" + '
  '0.004*"event" + 0.004*"export" + 0.004*"await" + 0.004*"async" + 0.004*"js" '
  '+ 0.004*"console" + 0.004*"user" + 0.004*"test" + 0.004*"value" + '
  '0.003*"login" + 0.003*"function" + 0.003*"router" + 0.003*"jest"'),
 (1,
  '0.009*"user" + 0.008*"id" + 0.007*"data" + 0.006*"request" + '
  '0.006*"password" + 0.006*"await" + 0.006*"name" + 0.005*"response" + '
  '0.005*"error" + 0.005*"async" + 0.005*"string" + 0.005*"type" + '
  '0.005*"query" + 0.004*"public" + 0.004*"json" + 0.004*"method" + '
  '0.004*"log" + 0.004*"const" + 0.004*"body" + 0.004*"username" + '
  '0.004*"email" + 0.004*"token" + 0.004*"result" + 0.004

In [ ]:
lda_model = model
def predict_unsupervised_tags(text):
    """
    Predict tags of a preprocessed text
    
    Args:
        text(list): preprocessed text
        
    Returns:
        relevant_tags(list): list of tags
    """
    
    corpus_new = id2word.doc2bow(text)
    topics = lda_model.get_document_topics(corpus_new)
    
    #find most relevant topic according to probability
    relevant_topic = topics[0][0]
    relevant_topic_prob = topics[0][1]
    
    for i in range(len(topics)):
        if topics[i][1] > relevant_topic_prob:
            relevant_topic = topics[i][0]
            relevant_topic_prob = topics[i][1]
            
    #retrieve associated to topic tags present in submited text
    potential_tags = lda_model.get_topic_terms(topicid=relevant_topic, topn=20)
    
    relevant_tags = [id2word[tag[0]] for tag in potential_tags if id2word[tag[0]] in text]
    
    return relevant_tags

for i in range(20):
    print(f"Liste de tags à prédire : {df.tags.iloc[i]}\n")
    print(f"Liste des tags prédits par le modèle non supervisé: {predict_unsupervised_tags(df.post.iloc[i])}\n\n")

# ajouter les sorties supervisées

Liste de tags à prédire : ['javascript']

Liste des tags prédits par le modèle non supervisé: ['id', 'data', 'name', 'response']


Liste de tags à prédire : ['python']

Liste des tags prédits par le modèle non supervisé: ['print', 'name', 'int']


Liste de tags à prédire : ['php', 'css']

Liste des tags prédits par le modèle non supervisé: ['file', 'version']


Liste de tags à prédire : ['c++']

Liste des tags prédits par le modèle non supervisé: ['std']


Liste de tags à prédire : ['javascript', 'reactjs']

Liste des tags prédits par le modèle non supervisé: ['react', 'const', 'app', 'page', 'log', 'route', 'useeffect', 'prop']


Liste de tags à prédire : ['java']

Liste des tags prédits par le modèle non supervisé: ['name', 'string', 'public', 'method']


Liste de tags à prédire : ['python']

Liste des tags prédits par le modèle non supervisé: ['file', 'installed', 'vscode', 'running']


Liste de tags à prédire : ['ios', 'flutter']

Liste des tags prédits par le modèle non supervisé: